In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np


# Load the CSV files
amplitude_spectrum = pd.read_csv('AMP_delta_theta_alpha_beta_LAHc7.csv', header=None)
decibels = pd.read_csv('PSD_DEC_delta_theta_alpha_beta_LAHc7.csv', header=None)
spectrum = pd.read_csv('PSD_delta_theta_alpha_beta_LAHc7.csv', header=None)
binary_labels = pd.read_csv('r_r_binary.csv')  # Ensure the binary labels match the samples

# Standardize each dataset
scaler = StandardScaler()

amplitude_spectrum_standardized = scaler.fit_transform(amplitude_spectrum)
decibels_standardized = scaler.fit_transform(decibels)
spectrum_standardized = scaler.fit_transform(spectrum)

# Convert binary labels to numpy array
binary_labels = binary_labels['Binary Label'].values  

X_train_full, X_test, y_train_full, y_test = train_test_split(
    amplitude_spectrum_standardized, binary_labels, test_size=0.2, random_state=42
)

2024-12-01 18:35:00.040518: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-01 18:35:00.266262: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-01 18:35:00.485522: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733096100.765579   10004 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733096100.902376   10004 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 18:35:01.599457: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
len(binary_labels)

253

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# Reserve a separate test set
X_train_full, X_test, y_train_full, y_test = train_test_split(
    amplitude_spectrum_standardized, binary_labels, test_size=0.2, random_state=42
)

# Verify the split
print(f"Training set shape (for k-fold): {X_train_full.shape}, {y_train_full.shape}")
print(f"Test set shape: {X_test.shape}, {y_test.shape}")


Training set shape (for k-fold): (202, 4), (202,)
Test set shape: (51, 4), (51,)


In [10]:
# Initialize K-Fold Cross-Validation
kf = KFold(n_splits=3, shuffle=True, random_state=42)

fold_accuracies = []
fold_losses = []

# Perform k-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
    print(f"\nFold {fold + 1}")
    
    # Split into training and validation sets for this fold
    X_train, X_val = X_train_full[train_idx], X_train_full[val_idx]
    y_train, y_val = y_train_full[train_idx], y_train_full[val_idx]
    
    # Build and compile the model
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = model.fit(X_train, y_train, epochs=20, batch_size=8, validation_data=(X_val, y_val), verbose=0)
    
    # Evaluate on validation set
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
    fold_losses.append(val_loss)
    fold_accuracies.append(val_accuracy)

# Calculate and display mean results from k-fold validation
mean_loss = np.mean(fold_losses)
mean_accuracy = np.mean(fold_accuracies)

print("\nK-Fold Validation Results:")
print(f"Mean Validation Loss: {mean_loss:.4f}")
print(f"Mean Validation Accuracy: {mean_accuracy:.4f}")


Fold 1
Validation Loss: 0.6948, Validation Accuracy: 0.4853

Fold 2
Validation Loss: 0.7161, Validation Accuracy: 0.4925

Fold 3
Validation Loss: 0.7037, Validation Accuracy: 0.5672

K-Fold Validation Results:
Mean Validation Loss: 0.7049
Mean Validation Accuracy: 0.5150


In [6]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("\nFinal Test Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")



Final Test Results:
Test Loss: 0.7152
Test Accuracy: 0.3529


In [24]:
import keras_tuner as kt

def build_model(hp):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(hp.Int('units', min_value=16, max_value=64, step=16), 
                              activation=hp.Choice('activation', ['relu', 'tanh'])),
        tf.keras.layers.Dense(hp.Int('units', min_value=16, max_value=64, step=16), 
                              activation=hp.Choice('activation', ['relu', 'tanh'])),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'sgd']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

In [25]:
# Initialize the tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=100,
    executions_per_trial=1,
    directory='my_dir',
    project_name='ffnn_tuning'
)

# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, verbose=1)

# Get the best hyperparameters and model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps.values}")

Reloading Tuner from my_dir/ffnn_tuning/tuner0.json
Best Hyperparameters: {'units': 16, 'activation': 'tanh', 'optimizer': 'sgd'}


In [12]:
import keras as kr
# Build the model with the best hyperparameters
final_model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='tanh', input_shape=(X_train.shape[1],)),  # First hidden layer
    tf.keras.layers.Dense(16, activation='tanh'),  # Second hidden layer
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer
])

optimizer = kr.optimizers.SGD(learning_rate=0.1)
# Compile the model with SGD optimizer
final_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = final_model.fit(
    X_train, y_train,
    epochs=20,  
    batch_size=8,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model on the test set
test_loss, test_accuracy = final_model.evaluate(X_test, y_test, verbose=0)
print(f"\nFinal Test Loss: {test_loss:.4f}")
print(f"Final Test Accuracy: {test_accuracy:.4f}")


Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.5404 - loss: 0.7352 - val_accuracy: 0.5926 - val_loss: 0.6829
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5982 - loss: 0.6586 - val_accuracy: 0.5926 - val_loss: 0.6871
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5650 - loss: 0.6694 - val_accuracy: 0.4815 - val_loss: 0.7058
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6277 - loss: 0.6589 - val_accuracy: 0.5185 - val_loss: 0.7167
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5276 - loss: 0.6828 - val_accuracy: 0.5185 - val_loss: 0.6782
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5945 - loss: 0.6725 - val_accuracy: 0.5926 - val_loss: 0.6684
Epoch 7/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4684 - loss: 0.6990 - val_accuracy: 0.4444 - val_loss: 0.7017
Epoch 8/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6587 - loss: 0.6369 - val_accuracy: 0.4815 - val_loss

In [15]:
# Predict probabilities for the test set
predicted_probabilities = final_model.predict(X_test)

# Convert probabilities to binary predictions (threshold: 0.5)
predicted_labels = (predicted_probabilities >= 0.5).astype(int).flatten()

# Compare predictions with actual labels
comparison_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': predicted_labels,
    'Probability': predicted_probabilities.flatten()
})

# Print a sample of the comparison
print(comparison_df.head(20))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
    Actual  Predicted  Probability
0        0          0     0.379378
1        1          1     0.693534
2        0          0     0.409110
3        0          1     0.700284
4        0          1     0.512581
5        0          0     0.303415
6        1          1     0.642386
7        1          0     0.348767
8        1          0     0.383353
9        0          0     0.309137
10       0          0     0.438354
11       1          0     0.351249
12       0          0     0.450367
13       1          0     0.265400
14       0          1     0.634236
15       0          1     0.705535
16       0          1     0.607997
17       1          1     0.606591
18       1          1     0.619339
19       0          0     0.407189


In [15]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

fold_accuracies = []
fold_losses = []

# Loop over each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_full)):
    print(f"\nFold {fold + 1}")

    # Split the training data into k-fold training and validation sets
    X_train, X_val = X_train_full[train_idx], X_train_full[val_idx]
    y_train, y_val = y_train_full[train_idx], y_train_full[val_idx]

    # Build the model
    final_model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='tanh', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(16, activation='tanh'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    # Compile the model with SGD optimizer
    optimizer = kr.optimizers.SGD(learning_rate=0.01)
    final_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    history = final_model.fit(
        X_train, y_train,
        epochs=5,
        batch_size=25,
        validation_data=(X_val, y_val),
        verbose=1
    )
    
    # Evaluate on the validation set
    val_loss, val_accuracy = final_model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    
    fold_losses.append(val_loss)
    fold_accuracies.append(val_accuracy)

# Calculate average performance across folds
mean_loss = np.mean(fold_losses)
mean_accuracy = np.mean(fold_accuracies)

print(f"\nMean Validation Loss (3-Fold): {mean_loss:.4f}")
print(f"Mean Validation Accuracy (3-Fold): {mean_accuracy:.4f}")



Fold 1
Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - accuracy: 0.4727 - loss: 0.7220 - val_accuracy: 0.4118 - val_loss: 0.7399
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4496 - loss: 0.7217 - val_accuracy: 0.4118 - val_loss: 0.7368
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4417 - loss: 0.7222 - val_accuracy: 0.3971 - val_loss: 0.7360
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.4693 - loss: 0.7189 - val_accuracy: 0.3824 - val_loss: 0.7352
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5023 - loss: 0.7082 - val_accuracy: 0.3824 - val_loss: 0.7334
Validation Loss: 0.7334, Validation Accuracy: 0.3824

Fold 2
Epoch 1/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.5658 - loss: 0.6967 - val_accuracy: 0.5224 - val_loss: 0.6896
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5380 - loss: 0.6981 - val_accuracy: 0.5075 - val_loss: 0.6916
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5